In [5]:
from html2image import Html2Image

hti = Html2Image(custom_flags=[
    '--default-background-color=00000000',
    '--hide-scrollbars',
    '--headless',
])
hti.screenshot(html_file='templates/template_coop.html', save_as='temp.png')

208582 bytes written to file /Users/tamino/Projects/hslu/dspro/dspro2-swiss-receipt-extractor/receipt_generation/temp.png


['/Users/tamino/Projects/hslu/dspro/dspro2-swiss-receipt-extractor/receipt_generation/temp.png']

In [6]:
from PIL import Image
import os

im = Image.open("temp.png")
im = im.crop(im.getbbox())

os.remove("temp.png")

In [7]:
import numpy as np
import numpy.random as rng

rng.seed(42)

image_array = np.array(im, dtype=np.float32)

background_color = np.array([255, 255, 255, 255])

# create a background canvas to generate a noise from
background = np.ones(image_array.shape)
background[:, :] = background_color

# generate a noise with floating point values representing the translucency of the background color
noise_dim = (image_array.shape[0], image_array.shape[1])
noise = rng.normal(-5, 20, noise_dim)
noise = np.round(np.clip(noise, 0, 50) / 50 * 255)

# apply noise as opacity to the background to create a noise in the color of the background
background[:, :, 3] = noise[:, :]

# add printing error lines
for _ in range(rng.randint(0, 6)):
    line_pos = rng.randint(0, image_array.shape[1] + 1)
    background[:, line_pos, :] = background_color

# convert noise overlay to an image
noise_overlay = Image.fromarray(background.astype(np.uint8))

In [8]:
# add the noise layer with transparency on top of the receipt image
noisy_image = Image.alpha_composite(im, noise_overlay)
noisy_image.save('out.png')

In [7]:
import cv2
import numpy as np

img = cv2.imread("out.png", cv2.IMREAD_COLOR)

width = img.shape[1]
height = img.shape[0]

pos_from = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
pos_to = np.float32([[200, 50], [600, 50], [50, 1000], [750, 1000]])

matrix = cv2.getPerspectiveTransform(pos_from, pos_to)
result = cv2.warpPerspective(img, matrix, (800, 1100))

cv2.imwrite('transformed.png', result)

(1882, 605, 3)


True